<h1> I. PREPARATION (Etat : 100% Terminé) </h1>

<h2>I.1. Importations </h2>

In [1]:
# Pour rendre les fichiers Excels exploitables après téléchargement
import os
import re                                # Module d'opérations à base d'expressions rationnelles
import shutil                            # Module High-level file operations
import tempfile                          # Module de génération de fichiers et répertoires temporaires
from fnmatch      import fnmatch         # Librairie de filtrage par motif des noms de fichiers UNIX
from os           import listdir         # Pour lister les fichiers et sous-dossiers existants dans un répertoire
from os.path      import isfile, join    # Manipulation des noms de répertoires communs
from zipfile      import ZipFile         # Manipulation des fichiers ZIP

# Pour pré-traiter, traiter, et post-traiter les données
import cartopy as ctp                               # Pour manipuler les données géographiques
import datetime as dt                               # Pour manipuler les données temporelles
import dateparser
import matplotlib.pyplot as plt                     # Pour représenter les données
import numpy as np                                  # Pour manipuler les données
import pandas as pd                                 # Pour manipuler les données en forme de DataFrame
import requests                                     # Pour récupérer les données HTTP

from bs4 import BeautifulSoup                       # Pour exploiter les données d'un site internet à base de code HTML
from sklearn.metrics import r2_score                # 
from sklearn import preprocessing                   #
from sklearn.linear_model import LassoCV            #
from sklearn.linear_model import LinearRegression   # Pour études de régression linéaire

# Pour analyser et optimiser le temps de traitement
import time
import threading
from tqdm.auto import tqdm

- <u>Variables :</u>
>- BOOL_DirExist : Boolean validating the existence of the sub-directory
>- DIR_Data : Data directory
>- DIR_Main : Main directory
>- DIR_XLSRank : Directory containing daily rank Excel file
>- EXT_RankFile : String of Daily rank Excel file extension
>- FILE : String of FileName.FileExt
>- LIST_FileID : ID of the file in string date format "AAAAMMDD_HHMMSS"
>- LIST_FilesTemp : Temporary file liste to compare
>- LIST_FilesToDwnld : Missing files list to download
>- LIST_RankDate : List of ranking dates format"AAAA-MM-DD HH:MM:SS"
>- LIST_SoupURLRank : List of daily ranks in the URL
>- LIST_Time : List of date as datetime.datetime 
>- SOUP_URLRank : Soup of the URL rank
>- URL_Ranks : URL of ranks
>- URL_DwnldRank : Download link of ranks Excel files (without file ID and .ext)

- <u>Functions (def):</u>
>- CHECK_DATA_DIR : Check that data directories exist
>- CHECK_FILE_LIST : Check that all excel ranking files exist in the subdirectory
>- CONVERT_TIME_FORMAT : Convert date format
>- DWNLD_DAILY_RANK_FILES : Download Excel ranking files
>- GET_RANKING_DATE_LIST : Extract list of ranking date in AAAA-MM-DD HH:MM:SS format
>- UPDATE_FORMAT : Resolve the problem to open downloaded Excel files


<h2>I.2. Définitions des répertoires, liens internet, et type de fichiers</h2>

In [2]:
# Directories & SubDirectories
DIR_Main = os.getcwd()        # Main directory
DIR_XLSRank = 'DataRank_2021' # Sub-directory containing xlsx daily rank files

#DataSkipperDir = 'DataSkipper_2021'

# Uniform Resource Locators (URL) of Vendee Globe 2020-2021 data
URL_Ranks = "https://www.vendeeglobe.org/fr/classement"
URL_DwnldRank = "https://www.vendeeglobe.org/download-race-data/vendeeglobe_"
URL_Skippers = "https://www.vendeeglobe.org/fr/glossaire"

# FilesType
EXT_RankFile = ".xlsx"

<h2>I.3. Outils de conversion </h2>

In [3]:
def CONVERT_TIME_FORMAT (LIST_URL_Time :'YYYY-MM-DD hh:mm:ss')\
      ->'datetime.datetime(YYYY,M,D,H,M),[YYYYMMDD_HHMMSS]':
    
    LIST_Time =[]               # Daily rank date list extracted from URL_Rank
    LIST_FileID = []            # File ID list
    for i in LIST_URL_Time:     # Converting all the daily rank date formats
        LIST_Time.append(dt.datetime.strptime(i, "%Y-%m-%d %H:%M:%S"))
        LIST_FileID.append(dt.datetime.strptime(i, "%Y-%m-%d %H:%M:%S").
                           strftime("%Y%m%d_%H%M%S"))
    return LIST_Time,LIST_FileID

<h2>I.4. Gestion des répertoires et des fichiers </h2>

In [4]:
def CHECK_DATA_DIR(DIR_Main:'Dir_String', TYPE_Data:'Type_String')\
      ->'BOOL_ExistDir, DIR_Data':
    
    DIR_Data = os.path.join(DIR_Main, TYPE_Data)
    if not os.path.exists(DIR_Data):   # Make subdirectory if doesn't exist
        os.mkdir(DIR_Data)
        BOOL_DirExist = False
    else:
        BOOL_DirExist = True           # Else pass
    print("The subdirectory exists "+TYPE_Data+" exists :"+\
          str(np.where(BOOL_DirExist==True,"OK","NOK")))
    return BOOL_DirExist, DIR_Data

In [5]:
def GET_RANKING_DATE_LIST(URL_Ranks : 'URL_String')\
      ->'List of ranking date in AAAA-MM-DD HH:MM:SS format':
    
    
    SOUP_URLRank = BeautifulSoup(requests.get(URL_Ranks).
                                 content.decode('utf-8'))            #Get Soup
    LIST_SoupURLRank = SOUP_URLRank\
          .findAll('option')                        #Extract only options list
    LIST_RankDate = []
    for iLIST_SoupURLRank in LIST_SoupURLRank:     #Generate ranking date list
        LIST_RankDate.append(iLIST_SoupURLRank.text[2:])
    LIST_RankDate = np.unique(LIST_RankDate[1:-1])        #Delete duplications
    print("Number of daily ranking files : " + str(len(LIST_RankDate)))
    return LIST_RankDate

In [6]:
def UPDATE_FORMAT(FILE : "FileName.FileExt string",
                  name_filter : "Name filer string",
                  change : 'string')\
      ->'Nothing : Excel files are updated to be workable':
    DIR_temp = tempfile.mkdtemp()
    try:
        NAME_temp = os.path.join(DIR_temp, 'new.zip')
        with ZipFile(FILE, 'r') as r, ZipFile(NAME_temp, 'w') as w:
            for item in r.infolist():
                DATA = r.read(item.filename)           
                DATA = change(DATA)
                w.writestr(item, DATA)
        shutil.move(NAME_temp, FILE)
    finally:
        shutil.rmtree(DIR_temp)

In [7]:
def DWNLD_DAILY_RANK_FILES(URL_DwnldRank : 'DownloadingURL_String',
                           LIST_FileID : 'List of string containing files ID',
                           EXT_RankFile : '.ext_file String',
                           DIR_Data : 'Data Directory string',
                           DIR_Main : 'Main Directory string')\
      -> 'Nothing : Excel files downloaded and saved in data directory':
    os.chdir(DIR_Data)                                    # Go to subdirectory
    with tqdm(range(len(LIST_FileID)),
              desc = "Downloading progress bar")\
              as outer :
        for iLIST_FileID in LIST_FileID:    # Download each Excel ranking file
            resp = requests.get(URL_DwnldRank+iLIST_FileID+EXT_RankFile)
            FILE = "vendeeglobe_"+iLIST_FileID+EXT_RankFile
            with open(FILE,'wb')as file:
                file.write(resp.content)
            UPDATE_FORMAT(FILE,name_filter='xl/styles.xml',
                          change=lambda d:re.sub(b'xxid="\d*"',b"",d))
            outer.update()
    os.chdir(DIR_Main)

In [8]:
def CHECK_FILE_LIST(DIR_Main: 'Main directory string',
                    DIR_Data: 'Ranking excel directory string',
                    LIST_FileID: 'List of excel file ID string')\
      -> 'List of string of excel file ID to download':  
    LIST_FilesTemp = [f for f in listdir(DIR_Data)\
                      if isfile(join(DIR_Data,f))]
    LIST_FilesToDwnld = []
    for iLIST_FilesTemp in LIST_FileID:
        if not str("vendeeglobe_"+iLIST_FilesTemp+".xlsx") in LIST_FilesTemp:
            LIST_FilesToDwnld.append(iLIST_FilesTemp)
    return LIST_FilesToDwnld

In [9]:
'''MAIN DOWNLOADING DATA'''

BOOL_DirExist, DIR_Data = CHECK_DATA_DIR(DIR_Main,DIR_XLSRank)
LIST_RankDate = GET_RANKING_DATE_LIST(URL_Ranks)
LIST_Time, LIST_FileID = CONVERT_TIME_FORMAT(LIST_RankDate)
"""if BOOL_DirExist == False:
    print('Subdirectory "Datarank_2021" complete : NOK')
    print('Downloading ', len(LIST_FileID),
          ' Excel daily ranking files')
    DWNLD_DAILY_RANK_FILES(URL_DwnldRank,
                           LIST_FileID,
                           EXT_RankFile,
                           DIR_Data,
                           DIR_Main)
else :"""
LIST_FilesToDwnld = CHECK_FILE_LIST(DIR_Main,
                                    DIR_Data,
                                    LIST_FileID)
if len(LIST_FilesToDwnld)!=0:
    print('Subdirectory "Datarank_2021" complete : NOK')
    print('Downloading ',len(LIST_FilesToDwnld),
          ' missing Excel daily ranking files')
    DWNLD_DAILY_RANK_FILES(URL_DwnldRank,
                          LIST_FilesToDwnld,
                          EXT_RankFile,
                          DIR_Data,
                          DIR_Main)
    
print("-------------------------------------------------")
print('Subdirectory "Datarank_2021" created : OK')
print('Subdirectory "Datarank_2021" complete : OK')

The subdirectory exists DataRank_2021 exists :OK
Number of daily ranking files : 702
-------------------------------------------------
Subdirectory "Datarank_2021" created : OK
Subdirectory "Datarank_2021" complete : OK


<h1>Chargement des données excel en DF (En cours)</h1>

In [10]:
def GEN_DAILY_RANKING_DF(DIR_Data,LIST_FileID,LIST_Time, DIR_Main):
    os.chdir(DIR_Data)
    print(os.getcwd())
    DF_Rank=pd.DataFrame()
    i=0
    for iLIST_FileID in LIST_FileID:
        #print(iLIST_FileID)
        DF_Temp = pd.read_excel("vendeeglobe_"+iLIST_FileID+EXT_RankFile,header=[3,4],nrows=33)
        #DF_Temp['Unnamed: 0_level_0']= dt.datetime.strftime(LIST_Time[i],"%Y/%m/%d %H:%M:%S")
        DF_Rank = pd.concat([DF_Rank, DF_Temp],ignore_index=True)
        i+=1
    
    return DF_Rank

In [11]:
DF_Rank = GEN_DAILY_RANKING_DF(DIR_Data,LIST_FileID,LIST_Time,DIR_Main)
DF_Rank.head()

D:\TELECOM PARIS\GitHub\Kit_BgD_AlexandreROULEAU\211103_FinalProject\DataRank_2021


DTF                 DTL Date d'arrivée\nArrival date  \
  Unnamed: 19_level_1 Unnamed: 20_level_1           Unnamed: 4_level_1   
0          24293.9 nm              0.0 nm                          NaN   
1          24294.2 nm              0.4 nm                          NaN   
2          24294.3 nm              0.5 nm                          NaN   
3          24294.5 nm              0.6 nm                          NaN   
4          24294.5 nm              0.6 nm                          NaN   

                                                            \
  Unnamed: 5_level_1 Unnamed: 6_level_1 Unnamed: 7_level_1   
0                NaN                NaN                NaN   
1                NaN                NaN                NaN   
2                NaN                NaN                NaN   
3                NaN                NaN                NaN   
4                NaN                NaN                NaN   

  Depuis 24 heures\nSince 24 hours                                             \
                      Cap\nHeading Distance\nDistance VMG\nVMG Vitesse\nSpeed   
0                             201°             6.1 nm  0.3 kts        0.3 kts   
1                             196°             6.0 nm  0.2 kts        0.3 kts   
2                             199°             5.5 nm  0.2 kts        0.2 kts   
3                             196°             5.6 nm  0.2 kts        0.2 kts   
4                             195°             5.8 nm  0.7 kts        0.8 kts   

   ... Temps de course\nRace time                                         \
   ...        Unnamed: 10_level_1 Unnamed: 11_level_1 Unnamed: 8_level_1   
0  ...                        NaN                 NaN                NaN   
1  ...                        NaN                 NaN                NaN   
2  ...                        NaN                 NaN                NaN   
3  ...                        NaN                 NaN                NaN   
4  ...                        NaN                 NaN                NaN   

                     Unnamed: 0_level_0         Écarts\nGaps  \
  Unnamed: 9_level_1 Unnamed: 0_level_1 Au premier\nTo first   
0                NaN                NaN                  NaN   
1                NaN                NaN                  NaN   
2                NaN                NaN                  NaN   
3                NaN                NaN                  NaN   
4                NaN                NaN                  NaN   

                                                    \
  Au premier\nTo first.1 Au précédent\nTo previous   
0                    NaN                       NaN   
1                    NaN                       NaN   
2                    NaN                       NaN   
3                    NaN                       NaN   
4                    NaN                       NaN   

                                                ►  
  Au précédent\nTo previous.1 Unnamed: 18_level_1  
0                         NaN                 NaN  
1                         NaN                 NaN  
2                         NaN                 NaN  
3                         NaN                 NaN  
4                         NaN                 NaN  

[5 rows x 38 columns]

In [12]:
pd.read_excel("vendeeglobe_20210127_170000.xlsx",header=[3,4],nrows=35).head()

Unnamed: 0_level_0         Rang\nRank  Nat. / Voile\nNat. / Sail  \
  Unnamed: 0_level_1 Unnamed: 1_level_1         Unnamed: 2_level_1   
0                NaN             1\nARV                   \nFRA 79   
1                NaN         Rang\nRank  Nat. / Voile\nNat. / Sail   
2                NaN                NaN                        NaN   
3                NaN                  2                   \nFRA 18   
4                NaN                  3                   \nMON 10   

                    Skipper / Bateau\nSkipper / crew  \
                                  Unnamed: 3_level_1   
0                              Charlie Dalin\nAPIVIA   
1                   Skipper / Bateau\nSkipper / crew   
2                                                NaN   
3                      Louis Burton\nBureau Vallée 2   
4  Boris Herrmann\nSeaexplorer - Yacht Club De Mo...   

  Date d'arrivée\nArrival date                                            \
            Unnamed: 4_level_1  Unnamed: 5_level_1    Unnamed: 6_level_1   
0                          NaN                 NaN                   NaN   
1                          NaN                 NaN                   NaN   
2            Heure FR\nHour FR  Latitude\nLatitude  Longitude\nLongitude   
3                   17:30 FR\n          46°03.01'N            04°00.21'W   
4                   17:30 FR\n          44°54.90'N            04°33.63'W   

                                       Temps de course\nRace time  \
                    Unnamed: 7_level_1         Unnamed: 8_level_1   
0               27/01/2021 21:35:47 FR        80j 06h 15min 47s\n   
1  Depuis 30 minutes\nSince 30 minutes                        NaN   
2                         Cap\nHeading             Vitesse\nSpeed   
3                                 114°                   18.4 kts   
4                                  35°                   20.7 kts   

                      ...                                                     \
  Unnamed: 9_level_1  ...                                Unnamed: 11_level_1   
0                NaN  ...                                                NaN   
1                NaN  ...  Depuis le dernier classement\nSince the the la...   
2           VMG\nVMG  ...                                       Cap\nHeading   
3           14.1 kts  ...                                               114°   
4           19.9 kts  ...                                                55°   

          Écarts\nGaps                                                   \
  Au premier\nTo first Au premier\nTo first.1 Au précédent\nTo previous   
0                  NaN                    NaN                       NaN   
1                  NaN                    NaN                       NaN   
2       Vitesse\nSpeed               VMG\nVMG        Distance\nDistance   
3             18.1 kts               16.2 kts                   54.1 nm   
4             13.6 kts               13.6 kts                   40.8 nm   

                                    Sur l'ortho\nOver ortho  \
        Au précédent\nTo previous.1          Vitesse\nSpeed   
0                               NaN                12.6 kts   
1  Depuis 24 heures\nSince 24 hours                     NaN   
2                      Cap\nHeading          Vitesse\nSpeed   
3                              104°                17.5 kts   
4                               76°                13.6 kts   

                                       ► Sur le fond\nOver ground  \
  Distance\nDistance Unnamed: 18_level_1           Vitesse\nSpeed   
0         24355.0 nm             119.6 %                 15.1 kts   
1                NaN                 NaN                      DTF   
2           VMG\nVMG  Distance\nDistance                      NaN   
3           17.1 kts            420.8 nm                  95.7 nm   
4           13.2 kts            325.2 nm                 149.4 nm   

                      
  Distance\nDistance  
0         29135.0 nm  
1                DT

In [13]:
test = pd.read_excel("vendeeglobe_20210127_170000.xlsx",header=[3,4],nrows=35).head()
test

Unnamed: 0_level_0         Rang\nRank  Nat. / Voile\nNat. / Sail  \
  Unnamed: 0_level_1 Unnamed: 1_level_1         Unnamed: 2_level_1   
0                NaN             1\nARV                   \nFRA 79   
1                NaN         Rang\nRank  Nat. / Voile\nNat. / Sail   
2                NaN                NaN                        NaN   
3                NaN                  2                   \nFRA 18   
4                NaN                  3                   \nMON 10   

                    Skipper / Bateau\nSkipper / crew  \
                                  Unnamed: 3_level_1   
0                              Charlie Dalin\nAPIVIA   
1                   Skipper / Bateau\nSkipper / crew   
2                                                NaN   
3                      Louis Burton\nBureau Vallée 2   
4  Boris Herrmann\nSeaexplorer - Yacht Club De Mo...   

  Date d'arrivée\nArrival date                                            \
            Unnamed: 4_level_1  Unnamed: 5_level_1    Unnamed: 6_level_1   
0                          NaN                 NaN                   NaN   
1                          NaN                 NaN                   NaN   
2            Heure FR\nHour FR  Latitude\nLatitude  Longitude\nLongitude   
3                   17:30 FR\n          46°03.01'N            04°00.21'W   
4                   17:30 FR\n          44°54.90'N            04°33.63'W   

                                       Temps de course\nRace time  \
                    Unnamed: 7_level_1         Unnamed: 8_level_1   
0               27/01/2021 21:35:47 FR        80j 06h 15min 47s\n   
1  Depuis 30 minutes\nSince 30 minutes                        NaN   
2                         Cap\nHeading             Vitesse\nSpeed   
3                                 114°                   18.4 kts   
4                                  35°                   20.7 kts   

                      ...                                                     \
  Unnamed: 9_level_1  ...                                Unnamed: 11_level_1   
0                NaN  ...                                                NaN   
1                NaN  ...  Depuis le dernier classement\nSince the the la...   
2           VMG\nVMG  ...                                       Cap\nHeading   
3           14.1 kts  ...                                               114°   
4           19.9 kts  ...                                                55°   

          Écarts\nGaps                                                   \
  Au premier\nTo first Au premier\nTo first.1 Au précédent\nTo previous   
0                  NaN                    NaN                       NaN   
1                  NaN                    NaN                       NaN   
2       Vitesse\nSpeed               VMG\nVMG        Distance\nDistance   
3             18.1 kts               16.2 kts                   54.1 nm   
4             13.6 kts               13.6 kts                   40.8 nm   

                                    Sur l'ortho\nOver ortho  \
        Au précédent\nTo previous.1          Vitesse\nSpeed   
0                               NaN                12.6 kts   
1  Depuis 24 heures\nSince 24 hours                     NaN   
2                      Cap\nHeading          Vitesse\nSpeed   
3                              104°                17.5 kts   
4                               76°                13.6 kts   

                                       ► Sur le fond\nOver ground  \
  Distance\nDistance Unnamed: 18_level_1           Vitesse\nSpeed   
0         24355.0 nm             119.6 %                 15.1 kts   
1                NaN                 NaN                      DTF   
2           VMG\nVMG  Distance\nDistance                      NaN   
3           17.1 kts            420.8 nm                  95.7 nm   
4           13.2 kts            325.2 nm                 149.4 nm   

                      
  Distance\nDistance  
0         29135.0 nm  
1                DT

In [73]:
File_test="vendeeglobe_20210305_080000.xlsx"
test = pd.read_excel(File_test,nrows=35).head(15)  #"vendeeglobe_20210127_210000.xlsx"
test

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 11,Unnamed: 12,Unnamed: 13,Unnamed: 14,Unnamed: 15,Unnamed: 16,Unnamed: 17,Unnamed: 18,Unnamed: 19,Unnamed: 20
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,Classement du vendredi 05 mars 2021 à 09h00 FR,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,Rang\nRank,Nat. / Voile\nNat. / Sail,Skipper / Bateau\nSkipper / crew,Date d'arrivée\nArrival date,NaN,NaN,NaN,Temps de course\nRace time,NaN,...,NaN,Écarts\nGaps,NaN,NaN,NaN,Sur l'ortho\nOver ortho,NaN,►,Sur le fond\nOver ground,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,Au premier\nTo first,NaN,Au précédent\nTo previous,NaN,Vitesse\nSpeed,Distance\nDistance,NaN,Vitesse\nSpeed,Distance\nDistance
4,NaN,1\nARV,\nFRA 17,Yannick Bestaven\nMaître Coq IV,NaN,NaN,NaN,28/01/2021 04:19:46 FR,80j 03h 44min 46s\n-10h 15min 00s,NaN,...,NaN,NaN,NaN,NaN,NaN,12.6 kts,24365.7 nm,117.3 %,14.8 kts,28583.8 nm
5,NaN,2\nARV,\nFRA 79,Charlie Dalin\nAPIVIA,NaN,NaN,NaN,27/01/2021 20:35:47 FR,80j 06h 15min 47s\n,NaN,...,NaN,NaN,02h 31min 01s,NaN,02h 31min 01s,12.6 kts,24365.7 nm,119.6 %,15.1 kts,29135.0 nm
6,NaN,3\nARV,\nFRA 18,Louis Burton\nBureau Vallée 2,NaN,NaN,NaN,28/01/2021 00:45:12 FR,80j 10h 25min 12s\n,NaN,...,NaN,NaN,06h 40min 26s,NaN,04h 09min 25s,12.6 kts,24365.7 nm,117.6 %,14.8 kts,28650.0 nm
7,NaN,4\nARV,\nFRA 01,Jean Le Cam\nYes we Cam !,NaN,NaN,NaN,28/01/2021 20:19:55 FR,80j 13h 44min 55s\n-16h 15min 00s,NaN,...,NaN,NaN,10h 00min 09s,NaN,03h 19min 43s,12.5 kts,24365.7 nm,112.9 %,14.1 kts,27501.5 nm
8,NaN,5\nARV,\nMON 10,Boris Herrmann\nSeaexplorer - Yacht Club De Mo...,NaN,NaN,NaN,28/01/2021 11:19:45 FR,80j 14h 59min 45s\n-06h 00min 00s,NaN,...,NaN,NaN,11h 14min 59s,NaN,01h 14min 50s,12.6 kts,24365.7 nm,116.8 %,14.7 kts,28448.5 nm
9,NaN,6\nARV,\nFRA 59,Thomas Ruyant\nLinkedOut,NaN,NaN,NaN,28/01/2021 05:42:01 FR,80j 15h 22min 01s\n,NaN,...,NaN,NaN,11h 37min 15s,NaN,22min 16s,12.6 kts,24365.7 nm,119.7 %,15.1 kts,29175.5 nm


In [51]:
test.loc[test['Unnamed: 8']=="Temps de course\nRace time"].index

Int64Index([2], dtype='int64')

In [52]:
test.loc[test["Unnamed: 5"]=="Latitude\nLatitude"].index

Int64Index([], dtype='int64')

In [53]:
not test.loc[test['Unnamed: 8']=="Temps de course\nRace time"].index.values

False

In [54]:
Head_Arrivee = test.loc[test['Unnamed: 8']=="Temps de course\nRace time"].index.values[0]+2
print(Head_Arrivee)
Head_EnCourse = test.loc[test["Unnamed: 5"]=="Latitude\nLatitude"].index.values[0]
print(Head_EnCourse)
test_Arrivee = pd.read_excel("vendeeglobe_20210127_210000.xlsx",header = [Head_Arrivee-1,Head_Arrivee], nrows=Head_EnCourse-Head_Arrivee-1)
test_Encourse = pd.read_excel("vendeeglobe_20210127_210000.xlsx",header = [Head_EnCourse, Head_EnCourse+1],nrows=33-(Head_EnCourse-Head_Arrivee-1))

IndexError: index 0 is out of bounds for axis 0 with size 0

In [74]:
RaceState = [False, False]
if test.loc[test["Unnamed: 5"]=="Latitude\nLatitude"].index.values:
    RaceState[0] = True
if test.loc[test['Unnamed: 8']=="Temps de course\nRace time"].index.values:
    RaceState[1] = True

print(RaceState)    
    
if RaceState == [True, False]:
    Head_EnCourse = test.loc[test["Unnamed: 5"]=="Latitude\nLatitude"].index.values[0]
    test_Encourse = pd.read_excel(File_test,header = [Head_EnCourse, Head_EnCourse+1],nrows=33)
    print(test_Encourse)
    
elif RaceState == [True,True]:
    Head_EnCourse = test.loc[test["Unnamed: 5"]=="Latitude\nLatitude"].index.values[0]
    Head_Arrivee = test.loc[test['Unnamed: 8']=="Temps de course\nRace time"].index.values[0]+2
    test_Arrivee = pd.read_excel(File_test,header = [Head_Arrivee-1,Head_Arrivee], nrows=Head_EnCourse-Head_Arrivee-1)
    test_Encourse = pd.read_excel(File_test,header = [Head_EnCourse, Head_EnCourse+1],nrows=33-(Head_EnCourse-Head_Arrivee-1))
    print(test_Encourse)
    print(test_Arrivee)
    
elif RaceState == [False, True]:
    Head_Arrivee = test.loc[test['Unnamed: 8']=="Temps de course\nRace time"].index.values[0]+2
    test_Arrivee = pd.read_excel(File_test,header = [Head_Arrivee-1,Head_Arrivee], nrows=33)
    print(test_Arrivee)

[False, True]
   Unnamed: 0_level_0         Rang\nRank Nat. / Voile\nNat. / Sail  \
   Unnamed: 0_level_1 Unnamed: 1_level_1        Unnamed: 2_level_1   
0                 NaN             1\nARV                  \nFRA 17   
1                 NaN             2\nARV                  \nFRA 79   
2                 NaN             3\nARV                  \nFRA 18   
3                 NaN             4\nARV                  \nFRA 01   
4                 NaN             5\nARV                  \nMON 10   
5                 NaN             6\nARV                  \nFRA 59   
6                 NaN             7\nARV                \nFRA 1000   
7                 NaN             8\nARV                  \nITA 34   
8                 NaN             9\nARV                  \nFRA 09   
9                 NaN            10\nARV                  \nFRA 53   
10                NaN            11\nARV                FR\nFRA 02   
11                NaN            12\nARV                  \nFRA 30   
12    

C:\Users\alexa\AppData\Local\Temp/ipykernel_14160/140249163.py:2: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if test.loc[test["Unnamed: 5"]=="Latitude\nLatitude"].index.values:


In [ ]:
test_Arrivee

In [ ]:
test_Encourse

<h1>Chargement des données de voiliers (90% terminé)</h1>

<u>A voir</u>

Voir pour l'indexage du DF :
- Par nom de skipper
- Par nom de bateau
- Par numéro de voile

Avis personnel : Numéro de voile probablement plus facilement identifiable pour les raisons suivantes:
- Longueur du string des numéro de voile courte. Référence unique (on peut changer de marin sur ce même bateau, ou donner un nouveau nom au voilier, ça ne changera pas la référence de la voile)
- Nom de skipper réduit pour certains (diminutif, ex:Alexandre --> Alex)
- Nom de bateau : long string (idem pour certains noms de marin), risque de différence élevé (donc plus difficile à merger)

In [ ]:
def GEN_SEPCS_DF (URL_Skipper):
    # Paramètres des types de données à extraire
    PARAM_Data = {
        "BoatSpecs" : ['ul', "class", "boats-list__popup-specs-list"],
        "Nom voilier" : ['h3',"class","boats-list__boat-name"],
        "Nom skipper" : ["span","class", "boats-list__skipper-name"]
    }
    
    # Fonction d'extraction de données techniques spécifiques
    #(BoatSpecs, BoatName, SkipperName) sous forme de list
    def LIST_SOUP (URL_Skipper, PARAM_Data,iPARAM_Data):
        return BeautifulSoup(requests.get(URL_Skipper).content\
                                 .decode("utf-8")).findAll(
                                    PARAM_Data[iPARAM_Data][0],
                                    {PARAM_Data[iPARAM_Data][1]:
                                     PARAM_Data[iPARAM_Data][2]})
    # Création d'un DF Spécifications techniques du voilier vierge
    DF_TechSpecs=pd.DataFrame()
    
    # Remplissage du DF
    for iPARAM_Data in PARAM_Data:
        LIST_Soup = LIST_SOUP (URL_Skipper, PARAM_Data, iPARAM_Data)
        # Remplir d'abord les données techniques du voilier
        #(Num. voile, Anciens nom, Architece, Chantier, Date lanc., Longueur,...)
        if iPARAM_Data == list(PARAM_Data.keys())[0]:
            idx = 0
            for iLIST_Soup in LIST_Soup:
                iDATA,iVAL = np.transpose([string.split(' : ',1) for string
                                           in iLIST_Soup.text.split("\n")[1:-1]])
                DF_TechSpecs = pd.concat([DF_TechSpecs,
                                          pd.DataFrame([iVAL],columns = iDATA)],
                                             ignore_index=True)
                idx+=1
        # Ajouter ensuite les colonnes Nom de voilier, et nom du marin;
        #puis les mettre en premières colonnes
        else:
            DF_TechSpecs[iPARAM_Data] = [string.text for string in LIST_Soup]
            DF_TechSpecs = DF_TechSpecs[DF_TechSpecs.columns.tolist()[-1:]+
                                        DF_TechSpecs.columns.tolist()[:-1]]
    
    # Homogénéisation des données
       # liste des valeurs [init, new] et des unités à supprimer
        #suivant le nom de la colonne 
    VAL_ToReplace = {"Date de lancement":[], # pour reconvertion des dates
                    "Anciens noms du bateau":[np.nan,''],"Voile quille":[np.nan,''], # Pour remplacement des valeurs
                    "Nombre de dérives":[["foiler", "2 asymétriques"],['foils', "2"]], # Pour remplacement des valeurs
                    "Longueur":[' m'],"Largeur" :[' m'],"Tirant d'eau":[' m'],"Hauteur mât":[' m'], # Suppresion des unités de longueur
                    "Surface de voiles au près":[' m2', ' m²'],"Surface de voiles au portant":[' m2', ' m²'], #Suppression des unités de surface
                    "Déplacement (poids)":[' t', ' tonnes']} #Suppresion des unités de masse
    for iCOL in VAL_ToReplace:
        if iCOL in ["Anciens noms du bateau","Voile quille","Nombre de dérives"]:      
            DF_TechSpecs[iCOL] = DF_TechSpecs[iCOL]\
                                    .replace(VAL_ToReplace[iCOL][0],
                                             VAL_ToReplace[iCOL][1])
        elif "Date de lancement" in iCOL:
            DF_TechSpecs[iCOL] = DF_TechSpecs[iCOL].apply(dateparser.parse)
        else :
            for iUNIT in VAL_ToReplace[iCOL]:
                DF_TechSpecs[iCOL] = DF_TechSpecs[iCOL].str.strip(iUNIT)
            if "Déplacement (poids)" in iCOL:
                DF_TechSpecs[iCOL] = DF_TechSpecs[iCOL].str\
                                        .replace('[a-zA-Z]', '0', regex=True)
            DF_TechSpecs[iCOL] = DF_TechSpecs[iCOL].str.replace(',','.')
            DF_TechSpecs[iCOL] = DF_TechSpecs[iCOL].astype('float')
    # Repositionnement des unités de dimensionnement dans l'entête de certaines colonnes
    TITLE_ToReplace = {"Longueur":'Longueur [m]',
                   "Largeur" :'Largeur m',
                   "Tirant d'eau":"Tirant d'eau [m]",
                   "Hauteur mât":'Hauteur mât [m]',
                   "Surface de voiles au près":'Surface de voiles au près [m²]',
                   "Surface de voiles au portant":'Surface de voiles au portant [m²]',
                   "Déplacement (poids)":'Déplacement (poids) [t]'}
    DF_TechSpecs = DF_TechSpecs.rename(columns = TITLE_ToReplace)
    return DF_TechSpecs

In [ ]:
DF_TechSpecs = GEN_SEPCS_DF(URL_Skippers)

DF_TechSpecs